<a href="https://colab.research.google.com/github/kobarmn/LangChain_GoogleColab/blob/main/test_OpenAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#pip install -q -U google-genai

In [16]:
pip install --upgrade openai

In [18]:
import json
from openai import OpenAI

def get_current_weather(location, unit="celsius"):
  weather_info = {
      "location": location,
      "temperature":"25",
      "unit":"celsius",
      "forecast":["sunny", "windy"],
  }
  return json.dumps(weather_info)

functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. Tokyo",
                },  # location <end>
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                },  # unit <end>
            },  # properties <end>
            "required": ["location"],
        },  # parameter <end>
    },
]

# 環境変数 (GEMINI_API_KEY) を設定すると、GeminiAPIライブラリを利用する際に、クライアントにより自動取得される。
# client = genai.Client()

client = OpenAI(
    api_key="APIKEY",
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

messages = [
    {"role":"user", "content":"What's the weather like in Tokyo"}
]

response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=messages,
    tools=functions
)

print(response)

BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'Invalid JSON payload received. Unknown name "name" at \'tools[0]\': Cannot find field.\nInvalid JSON payload received. Unknown name "description" at \'tools[0]\': Cannot find field.\nInvalid JSON payload received. Unknown name "parameters" at \'tools[0]\': Cannot find field.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'tools[0]', 'description': 'Invalid JSON payload received. Unknown name "name" at \'tools[0]\': Cannot find field.'}, {'field': 'tools[0]', 'description': 'Invalid JSON payload received. Unknown name "description" at \'tools[0]\': Cannot find field.'}, {'field': 'tools[0]', 'description': 'Invalid JSON payload received. Unknown name "parameters" at \'tools[0]\': Cannot find field.'}]}]}}]